# 오늘은 LeNet 구조를 만들어봅시다


LeNet 구조는 CNN이며, 초기에 만들어진 모델입니다. 

2가지 모델(Sigmoid, ReLU)를 만들어 두 모델의 성능을 비교해봅시다.


## 1.우선 필요 라이브러리를 import 합니다.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import datasets
import torchvision.transforms as transforms

import torch.optim as optim

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## 2. 딥러닝 모델을 설계할 때 활용하는 장비 확인

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 1.13.1+cpu  Device: cpu


## 3. MNIST 데이터 다운로드 

 1. Training data와 Test data 분리하기
 
 2. Training data를 Training data 와 Validation data로 분리하기

In [3]:
BATCH_SIZE = 64

transform = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,)),
    ])

train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

train, val = torch.utils.data.random_split(train_data, [int(0.8 * len(train_data)), int(0.2 * len(train_data))])


train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)


## 4. torch.nn을 이용하여 모델-1 만들기

   1) 아래의 그림 중 LeNet 구조를 구현 할 것
   
   2) Sigmoid 활성화 함수를 이용할 것
   
   
![](Comparison_image_neural_networks.svg.png)

In [4]:
import torch.nn as nn

class Model_1(nn.Module):
    def __init__(self):
        super(Model_1, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.sigmoid(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.sigmoid(x)
        x = self.pool2(x)
        x = x.view(-1, 16 * 5 * 5)
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        x = self.fc3(x)
        
        return x

## 5. torch.nn을 이용하여 모델-2 만들기

   LeNet 모델에서 ReLU 활성화 함수를 사용하시요

In [5]:
class Model_2(nn.Module):
    def __init__(self):
        super(Model_2, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.relu = nn.ReLU()
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool2(x)
        x = x.view(-1, 16 * 5 * 5)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        
        return x

## 7. 학습 준비하기

1) 1 epoch를 학습할 수 있는 함수 만들기

2) Test와 Validation data의 정확도 계산할 수 있는 함수 만들기

In [6]:
def training_epoch(train_loader, network, loss_func, optimizer, epoch):
    train_losses = []
    train_correct = 0
    log_interval = 300
    
    for batch_idx, (image, label) in enumerate(train_loader):
        image, label = image.to(device), label.to(device)

        # 미분값의 초기화
        optimizer.zero_grad()

        # Forward propagration 계산하기.
        outputs = network.forward(image)
        
        
        # Cross_entropy 함수를 적용하여 loss를 구하고 저장하기
        loss = loss_func(outputs, label)
        train_losses.append(loss.item())

        # training accuracy 정확도 구하기 위해 맞는 샘플 개수 세기
        pred = torch.argmax(outputs, dim=1)
        train_correct += pred.eq(label).sum()

        # Gradinet 구하기
        loss.backward()

        # weight값 update 하기
        optimizer.step()

        # 학습 상황 출력
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.2f}%)]\tLoss: {:.6f}'
                  .format(epoch, batch_idx * len(label), len(train_loader.dataset),100. * batch_idx / len(train_loader),
                          loss.item()))
            
    return train_losses, train_correct

In [7]:
def test_epoch(test_loader, network, loss_func, val = False):
    correct = 0
    
    test_losses = []
    
    with torch.no_grad():
        for batch_idx, (image, label) in enumerate(test_loader):
            image, label = image.to(device), label.to(device)

            # Forward propagration 계산하기.
            outputs = network.forward(image)

            # Cross_entropy 함수를 적용하여 loss를 구하기
            loss = loss_func(outputs, label)
            test_losses.append(loss.item())

            # Batch 별로 정확도 구하기
            pred = torch.argmax(outputs, dim=1)
            correct += pred.eq(label).sum()

        # 전체 정확도 구하기
        test_accuracy = 100. * correct / len(test_loader.dataset)

        #중간결과 출력
        if val is True:
                print('Validation set: Accuracy: {}/{} ({:.2f}%)\n'
              .format(correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
        
        else:
            print('Test set: Accuracy: {}/{} ({:.2f}%)\n'
                  .format(correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
        
    return test_losses, test_accuracy


## 8. 위 정의된 함수로 학습 함수 만들기

Adam Optimizer를 사용하여 학습시키기

In [8]:
def training(network, learning_rate = 0.001):
    
    epoches = 15
    
    cls_loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(network.parameters(), lr = learning_rate)
    
    train_losses_per_epoch = []
    test_losses_per_epoch = []
    
    train_accuracies = []
    test_accuracies = []
    
    
    for epoch in range(epoches):
                
        # 모델를 학습 중이라고 선언하기
        network.train()
        
        train_losses, train_correct = training_epoch(train_loader,network,cls_loss,optimizer, epoch)
        
        # epoch 별로 loss 평균값, 정확도 구하기
        average_loss = np.mean(train_losses)
        train_losses_per_epoch.append(average_loss)
        
        train_accuracy = train_correct / len(train_loader.dataset) * 100
        train_accuracies.append(train_accuracy)
        
        # epoch 별로 정확도 출력
        print('\nTraining set: Accuracy: {}/{} ({:.2f}%)'
              .format(train_correct, len(train_loader.dataset),100. * train_correct / len(train_loader.dataset)))

        
        ### 학습 중에 test 결과 보기
        
        # 모델 test 중인 것을 선언하기
        network.eval()
        
        correct = 0
        with torch.no_grad():
            test_losses, test_accuracy = test_epoch(val_loader, network, cls_loss, True)

        test_losses_per_epoch.append(np.mean(test_losses))
        test_accuracies.append(test_accuracy)
        
    with torch.no_grad():
        test_losses, test_accuracy = test_epoch(test_loader, network, cls_loss, False)
        
    return train_losses_per_epoch, test_losses_per_epoch, train_accuracies, test_accuracies


In [9]:
network = Model_1().to(device)
rlt_const = training(network)

Train Epoch: 0 [0/48000 (0.00%)]	Loss: 2.304011
Train Epoch: 0 [19200/48000 (40.00%)]	Loss: 0.845138
Train Epoch: 0 [38400/48000 (80.00%)]	Loss: 0.631324

Training set: Accuracy: 32162/48000 (67.00%)
Validation set: Accuracy: 10937/12000 (91.14%)

Train Epoch: 1 [0/48000 (0.00%)]	Loss: 0.348825
Train Epoch: 1 [19200/48000 (40.00%)]	Loss: 0.340111
Train Epoch: 1 [38400/48000 (80.00%)]	Loss: 0.201641

Training set: Accuracy: 44485/48000 (92.68%)
Validation set: Accuracy: 11238/12000 (93.65%)

Train Epoch: 2 [0/48000 (0.00%)]	Loss: 0.185105
Train Epoch: 2 [19200/48000 (40.00%)]	Loss: 0.146570
Train Epoch: 2 [38400/48000 (80.00%)]	Loss: 0.049101

Training set: Accuracy: 45624/48000 (95.05%)
Validation set: Accuracy: 11522/12000 (96.02%)

Train Epoch: 3 [0/48000 (0.00%)]	Loss: 0.158597
Train Epoch: 3 [19200/48000 (40.00%)]	Loss: 0.241910
Train Epoch: 3 [38400/48000 (80.00%)]	Loss: 0.127208

Training set: Accuracy: 46139/48000 (96.12%)
Validation set: Accuracy: 11599/12000 (96.66%)

Train Ep

In [10]:
network = Model_2().to(device)
rlt_const = training(network)

Train Epoch: 0 [0/48000 (0.00%)]	Loss: 2.304882
Train Epoch: 0 [19200/48000 (40.00%)]	Loss: 0.229798
Train Epoch: 0 [38400/48000 (80.00%)]	Loss: 0.080648

Training set: Accuracy: 43948/48000 (91.56%)
Validation set: Accuracy: 11612/12000 (96.77%)

Train Epoch: 1 [0/48000 (0.00%)]	Loss: 0.125944
Train Epoch: 1 [19200/48000 (40.00%)]	Loss: 0.029265
Train Epoch: 1 [38400/48000 (80.00%)]	Loss: 0.076984

Training set: Accuracy: 46758/48000 (97.41%)
Validation set: Accuracy: 11712/12000 (97.60%)

Train Epoch: 2 [0/48000 (0.00%)]	Loss: 0.028703
Train Epoch: 2 [19200/48000 (40.00%)]	Loss: 0.021141
Train Epoch: 2 [38400/48000 (80.00%)]	Loss: 0.053536

Training set: Accuracy: 47085/48000 (98.09%)
Validation set: Accuracy: 11754/12000 (97.95%)

Train Epoch: 3 [0/48000 (0.00%)]	Loss: 0.018225
Train Epoch: 3 [19200/48000 (40.00%)]	Loss: 0.132950
Train Epoch: 3 [38400/48000 (80.00%)]	Loss: 0.009621

Training set: Accuracy: 47274/48000 (98.49%)
Validation set: Accuracy: 11801/12000 (98.34%)

Train Ep

## 9. 두모델의 성능을 비교하시오

정답) 두 모델 모두 높은 정확도를 가지며 빠르게 학습이 진행된다. 하지만 최종적인 결과를 봤을 때, sigmoid를 사용한 모델은 98% 정도의 정확도에 그쳤지만, ReLU를 사용한 모델은 조금 더 높은 수치인 99%를 근소하게 넘는 정확도를 보이는 것으로 보아 ReLU를 사용한 모델이 더 성능이 좋다고 할 수 있다.